In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import TextClassificationPipeline

In [19]:
dfo = pd.read_csv('data/obama_cleaned.csv', dtype={'tweets': 'string', 'class': 'int8'})
dfo = dfo.rename(columns={'tweets' : 'text', 'class' : 'label'})
dfr = pd.read_csv('data/romney_cleaned.csv', dtype={'tweets': 'string', 'class': 'int8'})
dfr = dfr.rename(columns={'tweets' : 'text', 'class' : 'label'})

In [20]:
Xo = dfo['text']
yo = dfo['label'].map({1 : 2, 0 : 1, -1 : 0})
Xo, Xo_test, yo, yo_test = train_test_split(Xo, yo, test_size = 0.2)
Xo_train, Xo_eval, yo_train, yo_eval = train_test_split(Xo, yo, test_size = 0.25)
Xr = dfr['text']
yr = dfr['label'].map({1 : 2, 0 : 1, -1 : 0})
Xr, Xr_test, yr, yr_test = train_test_split(Xr, yr, test_size = 0.2)
Xr_train, Xr_eval, yr_train, yr_eval = train_test_split(Xr, yr, test_size = 0.25)


traindf_o = pd.concat([Xo_train,yo_train], axis = 1)
traindf_r = pd.concat([Xr_train,yr_train], axis = 1)
evaldf_o = pd.concat([Xo_eval,yo_eval], axis = 1)
evaldf_r = pd.concat([Xr_eval,yr_eval], axis = 1)
testdf_o = pd.concat([Xo_test,yo_test], axis = 1)
testdf_r = pd.concat([Xr_test,yr_test], axis = 1)

In [21]:
train_o = Dataset.from_pandas(traindf_o, split = 'train')
eval_o = Dataset.from_pandas(evaldf_o, split = 'eval')
test_o = Dataset.from_pandas(testdf_o, split = 'test')
train_r = Dataset.from_pandas(traindf_r, split = 'train')
eval_r = Dataset.from_pandas(evaldf_r, split = 'eval')
test_r = Dataset.from_pandas(testdf_r, split = 'test')

In [22]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train_o = train_o.map(tokenize_function, batched=True)
tokenized_eval_o = eval_o.map(tokenize_function, batched=True)
tokenized_test_o = test_o.map(tokenize_function, batched=True)
tokenized_train_r = train_r.map(tokenize_function, batched=True)
tokenized_eval_r = eval_o.map(tokenize_function, batched=True)
tokenized_test_r = test_r.map(tokenize_function, batched=True)

Map: 100%|██████████| 1130/1130 [00:00<00:00, 32058.30 examples/s]


In [23]:
model_o = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=3)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args_o = TrainingArguments(output_dir="checkpoints/test_trainer_o", evaluation_strategy="epoch", num_train_epochs=3)
trainer_o = Trainer(
    model=model_o,
    args=training_args_o,
    train_dataset=tokenized_train_o,
    eval_dataset=tokenized_eval_o,
    compute_metrics=compute_metrics,
)

trainer_o.train()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).









































































































































                                        
                                               

  0%|          | 0/1266 [19:41<?, ?it/s]         



{'eval_loss': 0.8569006323814392, 'eval_accuracy': 0.6693333333333333, 'eval_runtime': 15.7941, 'eval_samples_per_second': 71.229, 'eval_steps_per_second': 8.927, 'epoch': 1.0}


                                        
  0%|          | 0/1266 [20:23<?, ?it/s]          

{'loss': 0.8048, 'learning_rate': 3.0252764612954187e-05, 'epoch': 1.18}














































































































































                                        
                                               

  0%|          | 0/1266 [24:21<?, ?it/s]         



...

                                        
  0%|          | 0/1266 [26:02<?, ?it/s]           

{'loss': 0.5064, 'learning_rate': 1.0505529225908373e-05, 'epoch': 2.37}














































































































































                                        
                                                

  0%|          | 0/1266 [29:16<?, ?it/s]         

                                        
100%|██████████| 1266/1266 [13:13<00:00,  1.59it/s]

{'eval_loss': 1.144871473312378, 'eval_accuracy': 0.7155555555555555, 'eval_runtime': 19.0461, 'eval_samples_per_second': 59.067, 'eval_steps_per_second': 7.403, 'epoch': 3.0}
{'train_runtime': 793.8772, 'train_samples_per_second': 12.75, 'train_steps_per_second': 1.595, 'train_loss': 0.5932796532508886, 'epoch': 3.0}


TrainOutput(global_step=1266, training_loss=0.5932796532508886, metrics={'train_runtime': 793.8772, 'train_samples_per_second': 12.75, 'train_steps_per_second': 1.595, 'train_loss': 0.5932796532508886, 'epoch': 3.0})

In [24]:
trainer_o.save_model('roberta-models/model_obama')

In [25]:
model_r = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis", num_labels=3)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args_r = TrainingArguments(output_dir="checkpoints/test_trainer_r", evaluation_strategy="epoch", num_train_epochs=3)
trainer_r = Trainer(
    model=model_r,
    args=training_args_r,
    train_dataset=tokenized_train_r,
    eval_dataset=tokenized_eval_r,
    compute_metrics=compute_metrics,
)

trainer_r.train()









































































































































                                        
                                               

  0%|          | 0/1266 [1:03:41<?, ?it/s]       



{'eval_loss': 1.1323339939117432, 'eval_accuracy': 0.38133333333333336, 'eval_runtime': 16.0451, 'eval_samples_per_second': 70.115, 'eval_steps_per_second': 8.788, 'epoch': 1.0}


                                          
  0%|          | 0/1266 [1:04:22<?, ?it/s]        

{'loss': 1.0372, 'learning_rate': 3.0345911949685535e-05, 'epoch': 1.18}













































































































































                                          
                                               

  0%|          | 0/1266 [1:07:54<?, ?it/s]       



...

                                          
  0%|          | 0/1266 [1:09:28<?, ?it/s]         

{'loss': 0.9413, 'learning_rate': 1.069182389937107e-05, 'epoch': 2.36}












































































































































                                          
                                                

  0%|          | 0/1266 [1:12:33<?, ?it/s]       

                                          
100%|██████████| 1272/1272 [12:40<00:00,  1.67it/s]

{'eval_loss': 1.1442323923110962, 'eval_accuracy': 0.4497777777777778, 'eval_runtime': 16.0919, 'eval_samples_per_second': 69.911, 'eval_steps_per_second': 8.762, 'epoch': 3.0}
{'train_runtime': 760.4835, 'train_samples_per_second': 13.361, 'train_steps_per_second': 1.673, 'train_loss': 0.95439768137422, 'epoch': 3.0}


TrainOutput(global_step=1272, training_loss=0.95439768137422, metrics={'train_runtime': 760.4835, 'train_samples_per_second': 13.361, 'train_steps_per_second': 1.673, 'train_loss': 0.95439768137422, 'epoch': 3.0})

In [26]:
trainer_r.save_model('roberta-models/model_romney')

In [31]:
model_o = AutoModelForSequenceClassification.from_pretrained('roberta-models/model_obama')
model_r = AutoModelForSequenceClassification.from_pretrained('roberta-models/model_romney')

In [39]:
pipe_o = TextClassificationPipeline(model=model_o, tokenizer=tokenizer, return_all_scores=True)
pipe_r = TextClassificationPipeline(model=model_r, tokenizer=tokenizer, return_all_scores=True)

/Users/aarshpatel/Downloads/DMTM (CS 583)/Project 2/.venv/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [50]:
pipe_o('dfghj')[0][0]

{'label': 'negative', 'score': 0.17259548604488373}

In [55]:
pos = list()
neg = list()
neu = list()
pred_o = pd.DataFrame()
for t in testdf_o['text']:
    pred = pipe_o(t)
    neg.append(pred[0][0]['score'])
    neu.append(pred[0][1]['score'])
    pos.append(pred[0][2]['score'])
        
pred_o['pos'] = pos
pred_o['neg'] = neg
pred_o['neu'] = neu

[{'label': 'negative', 'score': 0.9284705519676208}, {'label': 'neutral', 'score': 0.06684950739145279}, {'label': 'positive', 'score': 0.004679996985942125}]
[{'label': 'negative', 'score': 0.0006533246487379074}, {'label': 'neutral', 'score': 0.001520446385256946}, {'label': 'positive', 'score': 0.9978261590003967}]
[{'label': 'negative', 'score': 0.995155930519104}, {'label': 'neutral', 'score': 0.002480592578649521}, {'label': 'positive', 'score': 0.0023635488469153643}]
[{'label': 'negative', 'score': 0.01624925062060356}, {'label': 'neutral', 'score': 0.9795315265655518}, {'label': 'positive', 'score': 0.004219228867441416}]
[{'label': 'negative', 'score': 0.9869658946990967}, {'label': 'neutral', 'score': 0.01121773011982441}, {'label': 'positive', 'score': 0.001816337346099317}]
[{'label': 'negative', 'score': 0.9947973489761353}, {'label': 'neutral', 'score': 0.00309565057978034}, {'label': 'positive', 'score': 0.0021069752983748913}]
[{'label': 'negative', 'score': 0.97998720

In [56]:
pos = list()
neg = list()
neu = list()
pred_r = pd.DataFrame()
for t in testdf_r['text']:
    pred = pipe_r(t)
    neg.append(pred[0][0]['score'])
    neu.append(pred[0][1]['score'])
    pos.append(pred[0][2]['score'])

pred_r['pos'] = pos
pred_r['neg'] = neg
pred_r['neu'] = neu

In [57]:
pred_o

,pos,neg,neu
0,0.004680,0.928471,0.066850
1,0.997826,0.000653,0.001520
2,0.002364,0.995156,0.002481
3,0.004219,0.016249,0.979532
4,0.001816,0.986966,0.011218
...,...,...,...
1120,0.031266,0.014823,0.953911
1121,0.002470,0.990746,0.006784
1122,0.012216,0.124009,0.863775
1123,0.004923,0.001688,0.993389


In [58]:
pred_o['class'] = list(yo_test.map({0 : -1, 1 : 0, 2 : 1}))

In [59]:
pred_r['class'] = list(yr_test.map({0 : -1, 1 : 0, 2 : 1}))

In [60]:
def pred_label(df):
    preds = list()
    for idx, row in df.iterrows():
        if row['pos'] >= row['neu'] and row['pos'] >= row['neg']:
            preds.append(1)
        elif row['neg'] >= row['neu'] and row['neg'] > row['pos']:
            preds.append(-1)
        elif row['neu'] > row['pos'] and row['neu'] > row['neg']:
            preds.append(0)
    df['pred'] = preds
    acc = accuracy_score(df['class'], df['pred'])
    prec = precision_score(df['class'], df['pred'], average = None, zero_division = np.nan)
    rec = recall_score(df['class'], df['pred'], average = None)
    f1 = f1_score(df['class'], df['pred'], average = None)
    print("Accuracy:", acc)
    print("Precision:", prec)
    print("Recall:", rec)
    print("F1:", f1)
    return df

In [61]:
predicted_labels_o = pred_label(pred_o)

Accuracy: 0.7057777777777777
Precision: [0.71590909 0.65527066 0.74550898]
Recall: [0.81185567 0.58673469 0.72173913]
F1: [0.76086957 0.61911171 0.73343152]


In [62]:
predicted_labels_r = pred_label(pred_r)

Accuracy: 0.5495575221238939
Precision: [0.6048906  0.39919355 0.4952381 ]
Recall: [0.80756014 0.29640719 0.24299065]
F1: [0.69168506 0.34020619 0.32601881]
